<a href="https://colab.research.google.com/github/hogch/masterproject_gan/blob/train-discriminator/SeqGAN_headlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Generation using GAN

This notebook generates news headlines using the Machine Learning technology GAN (Generative Adversarial Networks).

## Import dependencies



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/Masterproject
!ls

/content/drive/My Drive/Colab Notebooks/Masterproject
abcnews-date-text.csv	 news-headlines-short.db-journal
fake_data.txt		 positive_file.txt
headlines.csv		 real_data.txt
headlines-short.csv	 SeqGAN
imdb.pkl		 SeqGAN_headlines_dataloading_experiments.ipynb
news-headlines.db	 SeqGAN_headlines.ipynb
news-headlines-short.db


**Import required modules**

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

**Install required dependencies manually**

## Load Dataset

In [55]:
df = pd.read_csv('headlines-short.csv', sep=',', index_col='id')
print(df.shape)
df.head()

(10000, 3)


,publish_date,text,fake
id,,,
1,20030219,aba decides against community broadcasting lic...,0
2,20030219,act fire witnesses must be aware of defamation,0
3,20030219,a g calls for infrastructure protection summit,0
4,20030219,air nz staff in aust strike for pay rise,0
5,20030219,air nz strike to affect australian travellers,0


## Define Test-, Trainingset and Hyper-Parameter for the Discriminator

In [0]:
# Discriminator Hyper-Parameter
BATCH_SIZE = 64
VOCAB_SIZE = 16000 # 20
SEQ_LENGTH = 100
EMB_SIZE = 100
EMB_DIM = 64
FILTER_SIZES = [2,3]
NUM_CLASSES = 2
NUM_FILTERS = 50

In [73]:
texts = []
labels = []

for row in zip(df['text'], df['fake']):
  texts.append(row[0])
  labels.append(row[1])
  
dis_dataloader.load_train_data(texts, labels)

ValueError: ignored

In [0]:
from tflearn.datasets import imdb
from tflearn.data_utils import pad_sequences, to_categorical    
  
(X_train, y_train), (X_test, y_test), _ = imdb.load_data()
X_train = np.array(pad_sequences(X_train, maxlen=100))

X_test = np.array(pad_sequences(X_test, maxlen=100))

vocab_size = X_train.max() + 1
print('vocab size:', format(vocab_size))
y_train = to_categorical(np.array(y_train), 2)
y_test = np.array(y_test)

discriminator = Discriminator(vocab_size, SEQ_LENGTH, EMB_SIZE,
                              FILTER_SIZES, NUM_CLASSES, NUM_FILTERS)
discriminator.build_model()
discriminator.train(X_train, y_train, 5, BATCH_SIZE, .001)

vocab size: 99999
(64, 2)
(128, 2)
(192, 2)
(256, 2)
(320, 2)
(384, 2)
(448, 2)
(512, 2)
(576, 2)
(640, 2)
(704, 2)
(768, 2)
(832, 2)
(896, 2)
(960, 2)
(1024, 2)
(1088, 2)
(1152, 2)
(1216, 2)
(1280, 2)
(1344, 2)
(1408, 2)
(1472, 2)
(1536, 2)
(1600, 2)
(1664, 2)
(1728, 2)
(1792, 2)
(1856, 2)
(1920, 2)
(1984, 2)
(2048, 2)
(2112, 2)
(2176, 2)
(2240, 2)
(2304, 2)
(2368, 2)
(2432, 2)
(2496, 2)
(2560, 2)
(2624, 2)
(2688, 2)
(2752, 2)
(2816, 2)
(2880, 2)
(2944, 2)
(3008, 2)
(3072, 2)
(3136, 2)
(3200, 2)
(3264, 2)
(3328, 2)
(3392, 2)
(3456, 2)
(3520, 2)
(3584, 2)
(3648, 2)
(3712, 2)
(3776, 2)
(3840, 2)
(3904, 2)
(3968, 2)
(4032, 2)
(4096, 2)
(4160, 2)
(4224, 2)
(4288, 2)
(4352, 2)
(4416, 2)
(4480, 2)
(4544, 2)
(4608, 2)
(4672, 2)
(4736, 2)
(4800, 2)
(4864, 2)
(4928, 2)
(4992, 2)
(5056, 2)
(5120, 2)
(5184, 2)
(5248, 2)
(5312, 2)
(5376, 2)


In [0]:
class Dis_dataloader():
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.sentences = np.array([])
        self.labels = np.array([])

    def load_train_data(self, positive_file, negative_file):
        # Load data
        positive_examples = []
        negative_examples = []
        with open(positive_file)as fin:
            for line in fin:
                line = line.strip()
                line = line.split()
                parse_line = [int(x) for x in line]
                positive_examples.append(parse_line)
        with open(negative_file)as fin:
            for line in fin:
                line = line.strip()
                line = line.split()
                parse_line = [int(x) for x in line]
                if len(parse_line) == 20:
                    negative_examples.append(parse_line)
        self.sentences = np.array(positive_examples + negative_examples)

        # Generate labels
        positive_labels = [[0, 1] for _ in positive_examples]
        negative_labels = [[1, 0] for _ in negative_examples]
        self.labels = np.concatenate([positive_labels, negative_labels], 0)

        # Shuffle the data
        shuffle_indices = np.random.permutation(np.arange(len(self.labels)))
        self.sentences = self.sentences[shuffle_indices]
        self.labels = self.labels[shuffle_indices]

        # Split batches
        self.num_batch = int(len(self.labels) / self.batch_size)
        self.sentences = self.sentences[:self.num_batch * self.batch_size]
        self.labels = self.labels[:self.num_batch * self.batch_size]
        self.sentences_batches = np.split(self.sentences, self.num_batch, 0)
        self.labels_batches = np.split(self.labels, self.num_batch, 0)

        self.pointer = 0


    def next_batch(self):
        ret = self.sentences_batches[self.pointer], self.labels_batches[self.pointer]
        self.pointer = (self.pointer + 1) % self.num_batch
        return ret

    def reset_pointer(self):
        self.pointer = 0

## Discriminator
model for classifying sequences (here headlines) as real or fake.
In this implementation the discriminative model uses following layers: 
1.   embedding layer
2.   convolution layer
3.   max-pooling layer
4.   softmax layer

In [0]:
class Discriminator:
  def __init__(self, vocab_size, seq_length, emb_size, filter_sizes, num_classes, num_filters):
    self.vocab_size = vocab_size
    self.emb_size = emb_size
    self.seq_length = seq_length
    self.filter_sizes = filter_sizes
    self.num_classes = num_classes
    self.num_filters = num_filters

    self.X_input = tf.placeholder(tf.int32, shape=[None, self.seq_length], name='X_input')
    self.y_input = tf.placeholder(tf.float32, shape=[None, self.num_classes], name='y_input')
    #self.dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')
    
    # Keeping track of l2 regularization loss (optional)
    self.l2_reg_lambda = 0.0
    self.l2_loss = tf.constant(0.0)

  def build_model(self):
    self.embedding_layer = self.build_embedding_layer()
    self.convolution_maxpool_layer = self.build_convolution_maxpool_layer()
    self.scores, self.predictions = self.build_softmax_layer()
    
    self.calc_mean_cross_entropy_loss()
    self.calc_accuracy_and_cost()
        
  def build_embedding_layer(self):
    with tf.device('cpu:0'), tf.name_scope('embedding_layer'):
      W_emb = tf.Variable(
          initial_value=tf.random_uniform([self.vocab_size, self.emb_size], -1.0, 1.0), 
          name='W'
      )
      emb_chars = tf.nn.embedding_lookup(W_emb, self.X_input)
      self.emb_chars_expand = tf.expand_dims(emb_chars, -1)
    
  def build_convolution_maxpool_layer(self):
    pooled_outputs = []
    for filter_size in self.filter_sizes:
      with tf.name_scope('conv-maxpool-%s' % filter_size):
        # Convolution Layer
        filter_shape = [filter_size, self.emb_size, 1, self.num_filters]
        W_filters = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name='W')
        b = tf.Variable(tf.constant(0.1, shape=[self.num_filters]), name='b')
        conv = tf.nn.conv2d(
            input=self.emb_chars_expand,
            filter=W_filters,
            strides=[1,1,1,1],
            padding='VALID',
            name='conv'
        )
        # Apply non-linearity - activation function
        activation = tf.nn.relu(tf.nn.bias_add(conv, b), name='relu')
        # Maxpooling over outputs
        max_pooling = tf.nn.max_pool(
            value=activation,
            ksize=[1, self.seq_length-filter_size+1, 1, 1],
            strides=[1,1,1,1],
            padding='VALID',
            name='max_pooling'
        )
        pooled_outputs.append(max_pooling)
              
    self.num_filter_total = self.num_filters * len(self.filter_sizes)
    h_pool = tf.concat(pooled_outputs, axis=3)
    return tf.reshape(h_pool, [-1, self.num_filter_total])
        
  def build_softmax_layer(self):
    """with tf.name_scope('dropout'): # without highway function
      self.h_drop = tf.nn.dropout(
          self.convolution_maxpool_layer, 
          self.dropout_keep_prob
      )"""
    with tf.name_scope('softmax_output'):
      W_softmax = tf.Variable(
          tf.truncated_normal(
              [self.num_filter_total, self.num_classes], 
              stddev=0.1
          ), name='W'
      )
      b_softmax = tf.Variable(tf.constant(0.1, shape=[self.num_classes]), name='b')
            
      self.l2_loss += tf.nn.l2_loss(W_softmax)
      self.l2_loss += tf.nn.l2_loss(b_softmax)
      
      #self.scores = tf.nn.xw_plus_b(self.h_drop, W_softmax, b_softmax, name='scores')
      self.scores = tf.matmul(self.convolution_maxpool_layer, W_softmax) + b_softmax
      predictions = tf.argmax(self.scores, 1, name="predictions")
      
    return self.scores, predictions
  
  def calc_mean_cross_entropy_loss(self):
    with tf.name_scope('cross_entropy_loss'):
      losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.y_input)
      self.loss = tf.reduce_mean(losses) + self.l2_reg_lambda * self.l2_loss
      
  def calc_accuracy_and_cost(self):
    with tf.name_scope('accuracy'):
      correct_predictions = tf.equal(self.predictions, tf.argmax(self.y_input, 1))
      self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, 'float'), name='accuracy')
      self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.y_input))
  
  def train(self, X, y, num_epochs, batch_size, learning_rate):
    with tf.name_scope('loss'):
      optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
    
    with tf.Session() as sess:
      sess.run(tf.initialize_all_variables())
      epoch_i = 0
      
      while epoch_i < num_epochs:
        batch_i = 0
        batch_losses = []
        batch_accs = []
            
        for i in range(batch_size, X.shape[0], batch_size):
          X_batch, y_batch = X[batch_i:i], y[batch_i:i]
          print(y_batch.shape)
          feed_dict = {
              self.X_input: X_batch,
              self.y_input: y_batch
          }
          sess.run(optimizer, feed_dict)
          loss, accuracy = sess.run([self.cost, self.accuracy], feed_dict)
    
        time_str = datetime.datetime.now().isoformat()
        print("{}: step {}, loss {}, acc {}".format(time_str, i, str(self.loss), str(self.accuracy)))
        epoch_i += 1

## Build Discriminator model and Dataloader

In [0]:
discriminator = Discriminator(VOCAB_SIZE, SEQ_LENGTH, EMB_SIZE,
                              FILTER_SIZES, NUM_CLASSES, NUM_FILTERS)

discriminator.build_model()

dis_dataloader = Dis_dataloader(BATCH_SIZE)

### Pre-train and Train Discriminator model

In [0]:
discriminator.train(dis_dataloader, 5, BATCH_SIZE, .001)

vocab size:  99999
(0, 2)


TypeError: ignored